In [52]:
import pandas as pd
import torch

from sklearn.metrics import accuracy_score
from meat_dataset import MeatDataset
from transformers import BertTokenizer, BertForSequenceClassification

from utils import create_label_encoder

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Считывание данных из файлов

In [54]:
path = "model_data"
tokenizer = BertTokenizer.from_pretrained(path)
model = BertForSequenceClassification.from_pretrained(path, num_labels=6).to(device=device)

In [55]:
test_df = pd.read_csv("test.csv", delimiter=';')
X = test_df['text']
Y = test_df['mtype']
label_to_code, code_to_label = create_label_encoder(pd.unique(Y))
print(label_to_code)
print(code_to_label)

{'Баранина': 0, 'Говядина': 1, 'Индейка': 2, 'Кура': 3, 'Свинина': 4, 'Цыпленок': 5}
{0: 'Баранина', 1: 'Говядина', 2: 'Индейка', 3: 'Кура', 4: 'Свинина', 5: 'Цыпленок'}


# Вычисление метрик

In [56]:
def predict(text, device=torch.device('cpu')):
    item = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=512)
    item = {key: value.to(device=device) for key, value in item.items()}
    return code_to_label[model(**item).logits.argmax().item()]

In [57]:
predict(X[0], device)

'Цыпленок'

In [58]:
predicted = X.map(lambda text: predict(text, device))

In [59]:
print(f"Точность модели на тестовых данных - {accuracy_score(Y, predicted):.2f}")

Точность модели на тестовых данных - 0.96


# Работа модели на конкретных примерах из тестового задания

In [60]:
examples = ['Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг.',
            "Мясо премиум Предлагаем котлетное мясо мраморной говядины.",
            "спинка цб",
            'Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)\nСвободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.',
            'Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:',
            'Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.']

for example in examples:
    print(example)
    print(predict(example, device))
    print("_______")

Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг.
Свинина
_______
Мясо премиум Предлагаем котлетное мясо мраморной говядины.
Говядина
_______
спинка цб
Цыпленок
_______
Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)
Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.
Говядина
_______
Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:
Кура
_______
Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.
Говядина
_______
